# ETL LegalRES

* Extract: Bajar los JSON de legalRES de S3 
* Transform: Seleccionar las columnas que interesan, aplanar a una tabla y generar columnas nuevas
* Load: Cargar tabla limpia a S3

In [1]:
#!pip install awswrangler
#!pip install --upgrade pandas

In [14]:
import pandas as pd
import boto3
from tqdm.notebook import tqdm

import awswrangler as wr

import json
from pandas.io.json import json_normalize

#data = pd.read_json(data_location)

In [15]:
USE_LOCAL = True

In [16]:
bucket='legalres-storage-study-json'
data_key = 'AC000lvffws1.json'
data_location = 's3://{}/{}'.format(bucket, data_key)

session = boto3.Session(profile_name='legalbot')
boto3.setup_default_session(profile_name='legalbot')
s3 = boto3.client('s3')


Config datos ya extraidos para no duplicar

In [17]:
prefix_processed = 'processed'
data_location_processed = 's3://{}/{}/'.format(bucket, prefix_processed)

## Extract 

Datos ya extraidos para no duplicar

In [18]:
if USE_LOCAL:
    df_processed = pd.read_csv('../../data/processed/processed-studies.csv')
else:
    df_processed = wr.s3.read_parquet(path=data_location_processed,
                                  columns=['cve'], boto3_session=session )

537735

798060

880520

881320

In [19]:
df_processed.shape

(881320, 7)

In [20]:
if not USE_LOCAL:
    df_processed.to_csv('../../data/processed/processed-studies.csv', index=False)

In [21]:
cve_processed = list(df_processed['cve'].values)

In [22]:
assert len(cve_processed) == len(df_processed.index)

In [23]:
'AC0AEHf1alMr' in cve_processed

True

Prueba con un archivo para ver estructura

In [24]:
df_processed.head(2)

,rut,cve,fecha,lugar,tipo_sociedad,objeto,firmas
0,76305353-9,AC00DaeUV9O5,2013-10-16,"En PROVIDENCIA, Región METROPOLITANA DE SANTIA...",EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA,La Empresa tendrá por objeto desarrollar las s...,"[{'anotacion': None, 'firmado': 'Firmado elect..."
1,76313659-0,AC00iMWofV5q,2013-10-28,"En QUILPUE, Región de VALPARAISO, Chile, a 28 ...",EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA,La Empresa tendrá por objeto desarrollar las s...,"[{'anotacion': None, 'firmado': 'Firmado elect..."


In [25]:
data_stream = s3.get_object(Bucket=bucket,Key='AC00DaeUV9O5.json')['Body'].read().decode('utf-8')
data = json.loads(data_stream)
print(json.dumps(data, indent=4, ensure_ascii=False))

{
    "error_fatal": false,
    "disuelto": false,
    "rut": "76305353-9",
    "fecha": "2013-10-16",
    "cve": "AC00DaeUV9O5",
    "tipo_actuacion": "CONSTITUCION",
    "tipo_sociedad": "EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA",
    "estructura": [
        {
            "numero": 1,
            "tipo": "titulo_primero",
            "texto": "CONSTITUCION DE EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA SIGMAQ MAQUINARIA E INGENIERIA DIEGO IGNACIO ALVIAL ARREPOL E.I.R.L. Rut 76.305.353-9"
        },
        {
            "numero": 2,
            "tipo": "principal",
            "articulos": [
                {
                    "numero": 0,
                    "id": "articulo_0",
                    "articulo": "inicio",
                    "subtitulo": "",
                    "texto": "En PROVIDENCIA, Región METROPOLITANA DE SANTIAGO, Chile, a 16 de octubre del 2013, ante el Registro Electrónico de Empresas y Sociedades, comparece: DIEGO IGNACIO ALVIAL ARREPOL, Rut 14.374.

In [26]:
"\"tipo_actuacion\": \"CONSTITUCION\"" in data_stream[:200]

True

Helper functions

## Spot Fixes 

In [37]:
df = df_processed

In [38]:
df['tipo_actuacion'] = 'CONSTITUCION'

In [39]:
df.columns

Index(['rut', 'cve', 'fecha', 'tipo_actuacion', 'lugar', 'tipo_sociedad',
       'objeto', 'firmas'],
      dtype='object')

In [40]:
df = df[['rut', 'cve', 'fecha', 'tipo_actuacion', 'lugar', 'tipo_sociedad', 'objeto', 'firmas']]

In [41]:
df.head()

,rut,cve,fecha,tipo_actuacion,lugar,tipo_sociedad,objeto,firmas
0,76305353-9,AC00DaeUV9O5,2013-10-16,CONSTITUCION,"En PROVIDENCIA, Región METROPOLITANA DE SANTIA...",EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA,La Empresa tendrá por objeto desarrollar las s...,"[{'anotacion': None, 'firmado': 'Firmado elect..."
1,76313659-0,AC00iMWofV5q,2013-10-28,CONSTITUCION,"En QUILPUE, Región de VALPARAISO, Chile, a 28 ...",EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA,La Empresa tendrá por objeto desarrollar las s...,"[{'anotacion': None, 'firmado': 'Firmado elect..."
2,76313458-K,AC01CGsqBCMM,2013-10-25,CONSTITUCION,"En COYHAIQUE, Región de AYSEN DEL GRAL. CARLOS...",SOCIEDAD DE RESPONSABILIDAD LIMITADA,La Sociedad tendrá por objeto desarrollar las ...,"[{'anotacion': None, 'firmado': 'Firmado elect..."
3,76313026-6,AC01YjJkTvS3,2013-10-23,CONSTITUCION,"En PUENTE ALTO, Región METROPOLITANA DE SANTIA...",EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA,La Empresa tendrá por objeto desarrollar las s...,"[{'anotacion': None, 'firmado': 'Firmado elect..."
4,76304233-2,AC1DrSkE1pdl,2013-10-08,CONSTITUCION,"En TEMUCO, Región de LA ARAUCANIA, Chile, a 08...",EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA,La Empresa tendrá por objeto desarrollar las s...,"[{'anotacion': None, 'firmado': 'Firmado elect..."


### Standard changes 

In [42]:
df['fecha'] = pd.to_datetime(df['fecha'], format="%Y-%m-%d")

In [43]:
df['year'] = df['fecha'].dt.year
df['month'] = df['fecha'].dt.month

## Load 

En formato Apache Parquet

In [45]:
wr.s3.to_parquet(
    df,
    path='s3://{}/processed'.format(bucket),
    dataset=True,
    partition_cols=['year', 'month'],
    mode='overwrite'
)

{'paths': ['s3://legalres-storage-study-json/processed/year=2013/month=5/e6a0806da75045e7801c34eefdfd43e6.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=6/e6a0806da75045e7801c34eefdfd43e6.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=7/e6a0806da75045e7801c34eefdfd43e6.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=8/e6a0806da75045e7801c34eefdfd43e6.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=9/e6a0806da75045e7801c34eefdfd43e6.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=10/e6a0806da75045e7801c34eefdfd43e6.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=11/e6a0806da75045e7801c34eefdfd43e6.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=12/e6a0806da75045e7801c34eefdfd43e6.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2014/month=1/e6a0806da750